In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_raw = pd.read_csv("/kaggle/input/quora-question-pairs/train.csv.zip")

In [ ]:
new_df = df_raw.sample(20000)

In [ ]:
new_df.head()

In [ ]:
new_df.shape

In [ ]:
new_df.isnull().sum()

In [ ]:
new_df.duplicated().sum()

In [ ]:
df = new_df

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df.sample(10)

In [ ]:
df.info()

In [ ]:
print(df["is_duplicate"].value_counts())
print((df["is_duplicate"].value_counts()/df["is_duplicate"].count())*100)
df["is_duplicate"].value_counts().plot(kind="bar")

In [ ]:
# Repeated questions

qid = pd.Series(df['qid1'].tolist() + df['qid2'].tolist())
print('Number of unique questions',np.unique(qid).shape[0])
x = qid.value_counts()>1
print('Number of questions getting repeated',x[x].shape[0])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Repeated questions histogram

plt.hist(qid.value_counts().values,bins=160)
plt.yscale('log')
plt.show()

In [ ]:
df.head()

In [ ]:
que_df = df[["question1","question2"]]
que_df.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
#merging of text
questions = list(que_df["question1"]) + list(que_df["question2"])

In [ ]:
print(len(questions))
print(type(questions))

In [ ]:
questions[0:4]

In [ ]:
cv = CountVectorizer(max_features=3000) #taking only 2000 features
q1_arr,q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2) #splitting questions

In [ ]:
q1_arr[0:1]

In [ ]:
type(q1_arr)

In [ ]:
temp_df1 = pd.DataFrame(q1_arr,index=que_df.index)
temp_df2 = pd.DataFrame(q2_arr,index=que_df.index)

In [ ]:
temp_df1[0:2]

In [ ]:
temp_df1.shape

In [ ]:
#concating two dataframes
temp_df = pd.concat([temp_df1,temp_df2],axis = 1)

In [ ]:
temp_df.shape

In [ ]:
temp_df

In [ ]:
new_df.sample(2)

In [ ]:
temp_df["is_duplicate"] = new_df["is_duplicate"]

In [ ]:
temp_df.head()

In [ ]:
X = temp_df.iloc[:,0:-1]  #6000 columns are for 3000 of question 1 and 3000 of question 2
y = temp_df.iloc[:,-1]

In [ ]:
X

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf  = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

In [ ]:
np.array(y_train)
np.array(X_train)


In [ ]:
test_df = temp_df.loc[:,~temp_df.columns.duplicated()]

In [ ]:
duplicate_columns = test_df.columns[test_df.columns.duplicated()]
duplicate_columns

In [ ]:
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression()

# from xgboost import XGBClassifier
# xgb = XGBClassifier()
# xg.fit(X_train,y_train)
# y_pred = xgb.predict(X_test)
# accuracy_score(y_test,y_pred)

In [ ]:
lg.fit(X_train,y_train)

In [ ]:
y_pred = lg.predict(X_test)
accuracy_score(y_test,y_pred)

# Adding new features
1. q1 length
2. q2 length
3. q1 words
4. q2 words
5. words common 
6. words total
7. words share - words common/words total

In [ ]:
new_df.sample(2)

In [ ]:
#feature engineering
temp_df["q1_len"] = new_df["question1"].str.len()
temp_df["q2_len"] = new_df["question2"].str.len()

In [ ]:
temp_df.sample(3)

In [ ]:
temp_df["q1_num_words"] = new_df["question1"].apply(lambda row: len(row.split(" ")))
temp_df["q2_num_words"] = new_df["question2"].apply(lambda row: len(row.split(" ")))
temp_df.head()

In [ ]:
def total_words(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
    return (len(w1) + len(w2))

In [ ]:
temp_df['word_total'] = new_df.apply(total_words, axis=1)
temp_df.head()

In [ ]:
def common_words(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
    return len(w1 & w2)

In [ ]:
temp_df['word_common'] = new_df.apply(common_words, axis=1)
temp_df.head()

In [ ]:
temp_df['word_share'] = round(temp_df['word_common']/temp_df['word_total'],2)
temp_df.head()

In [ ]:
#Analysis of features
sns.displot(temp_df["q1_len"])
print("maximum num of characters : ",temp_df["q1_len"].max())
print("minimum num of characters : ",temp_df["q1_len"].min())
print("Average num of characters : ",int(temp_df["q1_len"].mean()))

In [ ]:
sns.displot(temp_df["q2_len"])
print("maximum num of characters : ",temp_df["q2_len"].max())
print("minimum num of characters : ",temp_df["q2_len"].min())
print("Average num of characters : ",int(temp_df["q2_len"].mean()))

In [ ]:
sns.displot(temp_df["q1_num_words"])
print("maximum num of words : ",temp_df["q1_num_words"].max())
print("minimum num of words : ",temp_df["q1_num_words"].min())
print("Average num of words : ",int(temp_df["q1_num_words"].mean()))

In [ ]:
sns.displot(temp_df["q2_num_words"])
print("maximum num of words : ",temp_df["q2_num_words"].max())
print("minimum num of words : ",temp_df["q2_num_words"].min())
print("Average num of words : ",int(temp_df["q2_num_words"].mean()))

In [ ]:
# common words
sns.distplot(temp_df[temp_df['is_duplicate'] == 0]['word_common'],label='non duplicate')
sns.distplot(temp_df[temp_df['is_duplicate'] == 1]['word_common'],label='duplicate')
plt.legend()
plt.show()

In [ ]:
# total words
sns.distplot(temp_df[temp_df['is_duplicate'] == 0]['word_total'],label='non duplicate')
sns.distplot(temp_df[temp_df['is_duplicate'] == 1]['word_total'],label='duplicate')
plt.legend()
plt.show()

In [ ]:
# word share
sns.distplot(temp_df[temp_df['is_duplicate'] == 0]['word_share'],label='non duplicate')
sns.distplot(temp_df[temp_df['is_duplicate'] == 1]['word_share'],label='duplicate')
plt.legend()
plt.show()

In [ ]:
temp_df.head()

In [ ]:
X = temp_df.drop(["is_duplicate"],axis=1)
y = temp_df["is_duplicate"]

In [ ]:
X

In [ ]:
y

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
X_train.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf  = RandomForestClassifier()


In [ ]:
rf.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import confusion_matrix,precision_score,recall_score

y_pred = rf.predict(X_test)
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(precision_score(y_test,y_pred))
print(recall_score(y_test,y_pred))


In [ ]:
from sklearn.linear_model import LogisticRegression
lg=LogisticRegression()

In [ ]:
lg.fit(X_train,y_train)

In [ ]:
y_pred = lg.predict(X_test)

In [ ]:
y_pred

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score
print(confusion_matrix(y_test,y_pred))
print(precision_score(y_test,y_pred))
print(recall_score(y_test,y_pred))

In [ ]:
print(f1_score(y_test,y_pred))